# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Establish csv file

city_weather_datafile = "../WeatherPy/output_data/city_weather_datafile.csv"

#Create dataframe from CSV file

city_weather_df = pd.read_csv(city_weather_datafile)

#Delete duplicate index column Unnamed: 0

del city_weather_df['Unnamed: 0']

city_weather_df.head()

,City,Country,Latitude,Longitude,Max Temp.,Humidity,Wind Speed,Cloudiness,Date,Hemisphere
0,Hilo,US,19.73,-155.09,78.80,65,4.70,90,1586395577,Northern Hemisphere
1,Dukat,RU,62.55,155.55,19.38,88,5.59,86,1586395728,Northern Hemisphere
2,Saint-Pierre,RE,-21.34,55.48,77.00,73,8.05,0,1586395729,Southern Hemisphere
3,Albany,US,42.60,-73.97,53.60,54,14.99,75,1586395731,Northern Hemisphere
4,Rikitea,PF,-23.12,-134.97,76.60,80,9.86,90,1586395732,Southern Hemisphere


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure Gmaps

gmaps.configure(api_key=g_key)

#Drop NaN values

city_weather_df = city_weather_df.dropna()

#Pull Latitude/Longitude and Humidity datapoints from dataframe and convert Humidity to float

lat_lng = city_weather_df[["Latitude","Longitude"]]
humidity = city_weather_df["Humidity"].astype(float)

#Create HeatMap Layer with Satellite format and maximum heat intensity equal to max humidity in dataset

fig=gmaps.figure()
fig = gmaps.figure(map_type='SATELLITE')

heatmap_humidity = gmaps.heatmap_layer(lat_lng,weights=humidity,dissipating=False,max_intensity=max(city_weather_df["Humidity"]),point_radius=2)

fig.add_layer(heatmap_humidity)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow Down City_Weather Dataframe based on preferable vacation weather conditions

ideal_vacation_df = pd.DataFrame(city_weather_df)

#Eliminate cities with Max Temp Greater than 85 degrees F

ideal_vacation_df = ideal_vacation_df.loc[ideal_vacation_df["Max Temp."]<=85]

#Eliminate cities with Max Temp Less than 70 Degrees F

ideal_vacation_df = ideal_vacation_df.loc[ideal_vacation_df["Max Temp."]>70]

#Eliminate cities with any Cloudiness

ideal_vacation_df = ideal_vacation_df.loc[ideal_vacation_df["Cloudiness"]==0]

#Eliminate any cities that have Wind Speed more than 10 MPH

ideal_vacation_df = ideal_vacation_df.loc[ideal_vacation_df["Wind Speed"]<=10]

#Drop any records with null values

ideal_vacation_df = ideal_vacation_df.dropna()

ideal_vacation_df
                                                                                                                                   

,City,Country,Latitude,Longitude,Max Temp.,Humidity,Wind Speed,Cloudiness,Date,Hemisphere
2,Saint-Pierre,RE,-21.34,55.48,77.00,73,8.05,0,1586395729,Southern Hemisphere
17,Guskhara,IN,23.50,87.75,77.45,31,4.50,0,1586395748,Northern Hemisphere
41,Mossamedes,AO,-15.20,12.15,75.49,51,8.03,0,1586395776,Southern Hemisphere
50,Adrar,MR,20.50,-10.07,79.92,14,9.55,0,1586395786,Northern Hemisphere
59,Oussouye,SN,12.48,-16.55,77.00,88,4.70,0,1586395795,Northern Hemisphere
77,Benguela,AO,-12.58,13.41,76.23,82,2.48,0,1586395815,Southern Hemisphere
174,Lanxi,CN,29.22,119.48,72.79,39,1.83,0,1586395938,Northern Hemisphere
178,Atar,MR,20.52,-13.05,76.08,16,6.06,0,1586395944,Northern Hemisphere
245,São Filipe,CV,14.90,-24.50,77.20,62,9.33,0,1586396042,Northern Hemisphere
325,Pochutla,MX,15.74,-96.47,82.18,65,8.14,0,1586396151,Northern Hemisphere


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
#Store info in hotel_df
hotel_df = ideal_vacation_df

#Create Hotel Name column
hotel_df["Hotel Name"] = ""
hotel_df

for index, row in hotel_df.iterrows():
    
# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
#Pull latitiude and longtude for each city
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    
        
    target_location = f"{lat},{lng}"
    
    params = {
         "location": target_location,
         "radius":5000,
         "keyword":"hotel",
         "type":"hotel",
         "key": g_key
    }   
        
    
    response = requests.get(base_url,params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest Hotel in {City},{Country} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except IndexError:
        print("No hotel found. Going to next record")

hotel_df.head()

No hotel found. Going to next record
No hotel found. Going to next record
No hotel found. Going to next record


,City,Country,Latitude,Longitude,Max Temp.,Humidity,Wind Speed,Cloudiness,Date,Hemisphere,Hotel Name
2,Saint-Pierre,RE,-21.34,55.48,77.00,73,8.05,0,1586395729,Southern Hemisphere,LODGE PALMAE
17,Guskhara,IN,23.50,87.75,77.45,31,4.50,0,1586395748,Northern Hemisphere,
41,Mossamedes,AO,-15.20,12.15,75.49,51,8.03,0,1586395776,Southern Hemisphere,Hotel ibis Styles Iu Namibe
50,Adrar,MR,20.50,-10.07,79.92,14,9.55,0,1586395786,Northern Hemisphere,
59,Oussouye,SN,12.48,-16.55,77.00,88,4.70,0,1586395795,Northern Hemisphere,Hotel Carabane
77,Benguela,AO,-12.58,13.41,76.23,82,2.48,0,1586395815,Southern Hemisphere,Aparthotel Mil Cidades
174,Lanxi,CN,29.22,119.48,72.79,39,1.83,0,1586395938,Northern Hemisphere,Shangri-La Hotel
178,Atar,MR,20.52,-13.05,76.08,16,6.06,0,1586395944,Northern Hemisphere,Odar kanawal
245,São Filipe,CV,14.90,-24.50,77.20,62,9.33,0,1586396042,Northern Hemisphere,Hotel Xaguate
325,Pochutla,MX,15.74,-96.47,82.18,65,8.14,0,1586396151,Northern Hemisphere,Hotel Alebrijes


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map

fig

Figure(layout=FigureLayout(height='420px'))